<https://github.com/PolymathicAI/xVal>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
import icecream
from icecream import ic

icecream.install()
ic_disable = False  # Global variable to disable ic
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)

In [3]:
import ast
import re
from flax.struct import dataclass
from datetime import datetime as dt
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import hephaestus as hp
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax.training import orbax_utils
import orbax.checkpoint
import orbax
import pandas as pd
from flax.training import train_state
from icecream import ic
from jax import random
from jax.tree_util import tree_flatten
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from hephaestus.models.time_series_decoder import TimeSeriesDS
from hephaestus.models.time_series_decoder import TimeSeriesConfig

pd.options.mode.copy_on_write = True

In [4]:
from transformers import FlaxBertModel, BertTokenizerFast
import jax.numpy as jnp

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = FlaxBertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Get the embeddings matrix
embeddings = model.params["embeddings"]["word_embeddings"]["embedding"]

# Now you can access specific embeddings like this:
# For example, to get embeddings for tokens 23, 293, and 993:
selected_embeddings = jnp.take(embeddings, jnp.array([23, 293, 993]), axis=0)

# If you want to get embeddings for specific words:
words = ["hello", "world", "example"]
tokens = tokenizer.convert_tokens_to_ids(words)
word_embeddings = jnp.take(embeddings, jnp.array(tokens), axis=0)
word_embeddings.shape

Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: {('pooler', 'dense', 'bias'), ('pooler', 'dense', 'kernel')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(3, 768)

In [5]:
embeddings[jnp.array([23, 293, 993])].shape

(3, 768)

In [6]:
def line2df(line, idx):
    data_rows = []
    line = ast.literal_eval(line)
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df.insert(0, "idx", idx)

    return df

In [7]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for idx, line in enumerate(tqdm(data)):
            dfs.append(line2df(line, idx))
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")


# Combine total mass of all planets into one column `planet<n>_m`
mass_regex = re.compile(r"planet(\d+)_m")
mass_cols = [col for col in df.columns if mass_regex.match(col)]
df["total_mass"] = df[mass_cols].sum(axis=1)

# Introduce categorical columns for the number of planets choose non null columns with mass
df["n_planets"] = df[mass_cols].notnull().sum(axis=1).astype("category")
# Create category acceleration if the sum of plane/d_[x,y, z] is greater than 0
df["acceleration_x"] = df[
    [col for col in df.columns if "planet" in col and "_x" in col]
].sum(axis=1)
# Set acceleration_x to "increasing" if greater than 0 else "decreasing"
df["acceleration_x"] = df["acceleration_x"].apply(
    lambda x: "increasing" if x > 0 else "decreasing"
)
df["acceleration_y"] = df[
    [col for col in df.columns if "planet" in col and "_y" in col]
].sum(axis=1)
df["acceleration_y"] = df["acceleration_y"].apply(
    lambda x: "increasing" if x > 0 else "decreasing"
)


df.describe()

,idx,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,...,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass
count,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,4.165044e+06,4.165044e+06,5.563957e+06,5.563957e+06,...,2.783627e+06,2.783627e+06,2.783627e+06,2.783627e+06,1.392864e+06,1.392864e+06,1.392864e+06,1.392864e+06,1.392864e+06,5.563957e+06
mean,6.248635e+04,9.748911e+00,-1.339198e-01,7.391138e-02,-1.340140e-01,7.291389e-02,-1.305344e-01,7.065633e-02,2.999306e+00,1.624756e+00,...,6.559150e-02,2.996303e+00,1.623874e+00,9.980576e-01,-1.276881e-01,6.519469e-02,3.002531e+00,1.625815e+00,1.001317e+00,1.049149e+01
std,3.607949e+04,5.993534e+00,1.228071e+00,1.213232e+00,1.227950e+00,1.212650e+00,1.217229e+00,1.203678e+00,1.157182e+00,5.876632e-01,...,1.200148e+00,1.153190e+00,5.270725e-01,5.764675e-01,1.211648e+00,1.199625e+00,1.156856e+00,5.167198e-01,5.779763e-01,3.991780e+00
min,0.000000e+00,0.000000e+00,-3.294763e+00,-2.997514e+00,-3.284004e+00,-2.998546e+00,-3.289790e+00,-2.998050e+00,1.000003e+00,1.000000e+00,...,-2.997621e+00,1.000054e+00,1.000000e+00,9.369537e-05,-3.273603e+00,-2.998913e+00,1.000103e+00,1.000000e+00,6.720938e-05,2.014597e+00
25%,3.124400e+04,4.655172e+00,-1.030131e+00,-9.020907e-01,-1.030516e+00,-9.028009e-01,-1.050662e+00,-9.211662e-01,1.993948e+00,1.000000e+00,...,-9.321272e-01,1.996853e+00,1.191548e+00,4.980967e-01,-1.071974e+00,-9.394428e-01,2.004240e+00,1.215927e+00,5.032645e-01,7.282371e+00
50%,6.249100e+04,9.523810e+00,-1.542335e-01,1.117099e-01,-1.538916e-01,1.099474e-01,-1.525520e-01,1.118031e-01,2.994477e+00,1.543047e+00,...,1.067355e-01,3.001879e+00,1.535683e+00,9.955040e-01,-1.507184e-01,1.055276e-01,3.000454e+00,1.519040e+00,1.003118e+00,1.028205e+01
75%,9.372800e+04,1.440000e+01,8.583344e-01,9.784987e-01,8.581045e-01,9.783998e-01,8.762358e-01,9.912902e-01,4.005747e+00,2.020672e+00,...,9.956064e-01,3.986406e+00,1.969644e+00,1.497746e+00,8.906780e-01,9.991220e-01,4.003332e+00,1.950518e+00,1.503457e+00,1.345633e+01
max,1.249990e+05,2.400000e+01,2.996370e+00,2.999014e+00,2.993319e+00,2.999536e+00,2.990464e+00,2.998478e+00,4.999994e+00,2.999984e+00,...,3.000881e+00,4.999990e+00,2.999909e+00,1.999957e+00,2.985180e+00,2.998936e+00,4.999679e+00,2.999497e+00,1.999999e+00,2.382455e+01


In [8]:
# Get train test split at 80/20
time_series_config = TimeSeriesConfig.generate(df=df)
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
# del df
train_ds = TimeSeriesDS(train_df, time_series_config)
test_ds = TimeSeriesDS(test_df, time_series_config)
len(train_ds), len(test_ds)

(99999, 25001)

In [9]:
def make_batch(ds: TimeSeriesDS, start: int, length: int):
    numeric = []
    categorical = []
    for i in range(start, length + start):
        numeric.append(ds[i][0])
        categorical.append(ds[i][1])
    # print index of None values
    return {"numeric": jnp.array(numeric), "categorical": jnp.array(categorical)}


batch = make_batch(train_ds, 0, 4)
# batch

In [10]:
multiplier = 4
time_series_regressor = hp.time_series_decoder.SimplePred(
    time_series_config, d_model=512, n_heads=8 * multiplier
)

In [11]:
key = random.PRNGKey(0)
init_key, dropout_key = random.split(key)
vars = time_series_regressor.init(
    {"params": init_key, "dropout": dropout_key},
    batch["numeric"],
    categorical_inputs=batch["categorical"].astype(jnp.int32),
    deterministic=False,
)
dropout_key, original_dropout_key = random.split(dropout_key)

In [12]:
df.shape

(5563957, 31)

In [13]:
# ic.disable()

In [14]:
x = time_series_regressor.apply(
    vars,
    batch["numeric"],
    batch["categorical"].astype(jnp.int32),
    deterministic=False,
    rngs={"dropout": dropout_key},
)
print(x.get("numeric_out").shape)
# Check if categorical input is None and print None or it's shape
print(x.get("categorical_out").shape if x.get("categorical_out") is not None else None)

(4, 27, 59)
(4, 2, 59, 38)


In [16]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters and total
    number of parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes


def count_parameters(params):
    return sum(jnp.prod(jnp.array(p.shape)) for p in jax.tree_util.tree_leaves(params))


mem = calculate_memory_footprint(vars)
total_params = count_parameters(vars)


print(f"Memory of custom: {mem / 1e6:.2f} MB with {total_params:,} parameters")

Memory of custom: 141.42 MB with 35,355,005 parameters


In [17]:
# ic.disable()

In [18]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)

mask_data = False


def clip_gradients(gradients, max_norm):
    total_norm = jnp.sqrt(sum(jnp.sum(jnp.square(grad)) for grad in gradients.values()))
    scale = max_norm / (total_norm + 1e-6)
    clipped_gradients = jax.tree_map(
        lambda grad: jnp.where(total_norm > max_norm, grad * scale, grad), gradients
    )
    return clipped_gradients


def add_time_shifts(inputs: jnp.array, outputs: jnp.array) -> jnp.array:
    inputs_offset = 1
    inputs = inputs[:, :, inputs_offset:]
    tmp_null = jnp.full((inputs.shape[0], inputs.shape[1], inputs_offset), jnp.nan)
    inputs = jnp.concatenate([inputs, tmp_null], axis=2)
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)
    # Add ext
    if outputs.ndim == inputs.ndim + 1:
        nan_mask_expanded = jnp.expand_dims(nan_mask, axis=-1)
        nan_mask_expanded = jnp.broadcast_to(nan_mask_expanded, outputs.shape)
    else:
        nan_mask_expanded = nan_mask

    # Apply mask to outputs
    outputs = jnp.where(nan_mask_expanded, jnp.zeros_like(outputs), outputs)

    return inputs, outputs, nan_mask
    outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    return inputs, outputs, nan_mask


def numeric_loss(inputs, outputs):
    inputs, outputs, nan_mask = add_time_shifts(inputs, outputs)
    # TODO make loss SSL for values greater than 0.5 and MSE for values less than 0.5
    raw_loss = jnp.abs(outputs - inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    loss = masked_loss.sum() / (~nan_mask).sum()
    return loss


def categorical_loss(inputs, outputs):

    inputs, outputs, nan_mask = add_time_shifts(inputs, outputs)
    inputs = inputs.astype(jnp.int32)
    raw_loss = optax.softmax_cross_entropy_with_integer_labels(outputs, inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss).mean()
    return masked_loss


def base_loss(
    numeric_inputs,
    categorical_inputs,
    outputs,
):
    numeric_out = outputs["numeric_out"]
    categorical_out = outputs["categorical_out"]
    numeric = numeric_loss(numeric_inputs, numeric_out)
    categorical = categorical_loss(categorical_inputs, categorical_out)
    return numeric + categorical


def base_loss_old(inputs, outputs):
    """TODO HERE IS THE SHIT"""
    # Remove the first value and add a jnp.nan to the end
    # inputs = inputs * 3
    inputs_offset = 1
    inputs = inputs[:, :, inputs_offset:]
    # Add a jnp.nan to the end
    temp_null = jnp.full((inputs.shape[0], inputs.shape[1], inputs_offset), jnp.nan)
    inputs = jnp.concatenate([inputs, temp_null], axis=2)
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)

    # outputs = outputs[:, :, :-inputs_offset]
    outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    # raw_loss = optax.squared_error(outputs, inputs)
    # compute manually
    # raw_loss = jnp.square(outputs - inputs)
    # Abs loss
    raw_loss = jnp.abs(outputs - inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    loss = masked_loss.sum() / (~nan_mask).sum()

    return loss


def calculate_loss(
    params,
    state,
    numeric_inputs,
    categorical_inputs,
    dropout_key,
    mask_data: bool = True,
):
    outputs = state.apply_fn(
        {"params": params},
        # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs.astype(jnp.int32),
        rngs={"dropout": dropout_key},
        deterministic=False,
        mask_data=mask_data,
    )
    loss = base_loss(
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs,
        outputs=outputs,
    )
    # Create mask for nan inputs

    return loss


@jax.jit
def train_step(
    state: train_state.TrainState,
    numeric_inputs,
    categorical_inputs,
    base_key,
    # mask_data=True,
):
    # print("In train step")
    dropout_key, mask_key, new_key = jax.random.split(
        base_key, 3
    )  # TODO Figure out mask key
    # print("Making masks")
    # numeric_inputs = batch["numeric"]
    # categorical_inputs = batch["categorical"]
    # print("Made masks")

    def calculate_loss_with_mask(params):
        return calculate_loss(
            params,
            state,
            numeric_inputs=numeric_inputs,
            categorical_inputs=categorical_inputs,
            dropout_key=dropout_key,
            mask_data=True,
        )

    def loss_fn(params):
        return calculate_loss_with_mask(params)

    # def calculate_loss_without_mask(params):
    #     return calculate_loss(params, state, batch, dropout_key, mask_data=False)

    # def loss_fn(params):
    #     return jax.lax.cond(
    #         mask_data,
    #         lambda _: calculate_loss_with_mask(params),
    #         lambda _: calculate_loss_without_mask(params),
    #         operand=None,
    #     )

    # def loss_fn(params):
    #     return calculate_loss(params, state, batch, dropout_key, mask_data=mask_data)

    grad_fn = jax.value_and_grad(loss_fn)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, new_key


def evaluate(params, state, inputs, mask_data: bool = True):
    outputs = state.apply_fn(
        {"params": params},
        # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
        inputs,
        deterministic=True,
        mask_data=mask_data,
    )
    loss = base_loss(inputs, outputs)
    return loss


@jax.jit
def eval_step(
    state: train_state.TrainState, numeric_inputs, categorical_inputs, base_key
):
    # mask_data=True
    mask_key, dropout_key, new_key = jax.random.split(base_key, 3)

    def calculate_loss_with_mask(params):
        return calculate_loss(
            params,
            state,
            numeric_inputs=numeric_inputs,
            categorical_inputs=categorical_inputs,
            dropout_key=dropout_key,
            mask_data=True,
        )

    def calculate_loss_without_mask(params):
        return calculate_loss(
            params, state=state, batch=batch, dropout_key=dropout_key, mask_data=False
        )

    def loss_fn(params):
        return calculate_loss_with_mask(params)

    # TODO Reimplement this...
    # def loss_fn(params):
    #     return jax.lax.cond(
    #         mask_data,
    #         lambda _: calculate_loss_with_mask(params),
    #         lambda _: calculate_loss_without_mask(params),
    #         operand=None,
    #     )

    # def loss_fn(params):
    #     return evaluate(params, state, batch, mask_data=mask_data)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss = loss_fn(state.params)
    return loss, new_key


def create_train_state(model, prng, batch, lr):
    init_key, dropout_key = random.split(prng)
    params = model.init(
        {"params": init_key, "dropout": dropout_key},
        batch["numeric"],
        batch["categorical"],
        deterministic=False,
    )
    # optimizer = optax.chain(optax.adam(lr))
    optimizer = optax.chain(optax.clip_by_global_norm(0.4), optax.adam(lr))
    # optimizer_state = optimizer.init(params)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params["params"],
        tx=optimizer,
        # tx_state=optimizer_state,
    )


batch_size = 2
# batch = train_ds[0]
# state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

In [19]:
writer_name = "BERT_Embeddings"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
train_summary_writer = SummaryWriter("runs/" + model_name)

MASK_DATA = True

test_set_key = random.PRNGKey(4454)

batch_size = 16
train_data_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

# train_data_loader = DataLoader(train_ds, batch_size=256 // 2, shuffle=True)
# test_data_loader = DataLoader(test_ds, batch_size=256 // 2, shuffle=True)

batch_count = 0
base_key = random.PRNGKey(42)

# Disable IC for training
max_iters = 200
ic.disable()
for j in trange(1, desc=f"epochs for {train_summary_writer.log_dir}"):
    # arrs = train_data_loader()
    for i in tqdm(train_data_loader, leave=False, desc="batches"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        # batch = make_batch(train_ds, i[0], 4)
        state, loss, base_key = train_step(
            state,
            jnp.array(i[0]),
            jnp.array(i[1]),
            base_key,
            # mask_data=MASK_DATA,
        )
        if jnp.isnan(loss):
            raise ValueError("Nan Value in loss, stopping")
        batch_count += 1

        if batch_count % 1 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
        if batch_count % 10 == 0:
            numeric_eval, categorical_eval = next(iter(test_data_loader))
            test_loss, base_key = eval_step(
                state,
                jnp.array(numeric_eval),
                jnp.array(categorical_eval),
                base_key,
                # mask_data=MASK_DATA,
            )
            train_summary_writer.add_scalar(
                "loss/test_loss", np.array(test_loss.item()), batch_count
            )
            train_summary_writer.flush()
        # if batch_count > 200:
        #     break
        if batch_count > max_iters:
            break

train_summary_writer.close()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


epochs for runs/2024-09-02T21:27:48BERT_Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

batches:   0%|          | 0/6250 [00:00<?, ?it/s]

In [ ]:
train_ds[0][0]

array([[ 0.        ,  0.46511628,  0.93023256, ...,         nan,
                nan,         nan],
       [ 1.56006022,  1.68985799,  1.75358875, ...,         nan,
                nan,         nan],
       [-0.85443699, -0.5143588 , -0.15420858, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [ 6.97412249,  6.97412249,  6.97412249, ...,         nan,
                nan,         nan]])

In [ ]:
state.apply_fn(
    {"params": state.params},
    # jnp.array(i[0]),
    # jnp.array(i[1]),
    jnp.array([train_ds[0][0]]),
    jnp.array([train_ds[0][1]]),
    deterministic=True,
    mask_data=False,
)

{'numeric_out': Array([[[19.572609  , 19.748047  , 19.981857  , ..., 20.350695  ,
          20.330154  , 20.339558  ],
         [ 0.12221669,  0.11699589,  0.11444642, ..., -0.45419967,
          -0.44938296, -0.4528414 ],
         [ 1.2045583 ,  1.2037084 ,  1.2043195 , ...,  0.20396292,
           0.20125163,  0.20699549],
         ...,
         [ 1.5665928 ,  1.5792354 ,  1.5928653 , ...,  1.1835138 ,
           1.1834472 ,  1.1980616 ],
         [ 0.8753089 ,  0.8777581 ,  0.89333886, ...,  0.8083393 ,
           0.8210919 ,  0.83338034],
         [ 3.3444226 ,  3.3523316 ,  3.385091  , ...,  3.3032634 ,
           3.3177614 ,  3.3455076 ]]], dtype=float32),
 'categorical_out': Array([[[[ 1.2840094e-01,  3.6852699e-02,  2.2886926e-01, ...,
            5.2909237e-01, -6.6878214e-03, -6.4119155e-04],
          [ 1.2872720e-01,  3.6852699e-02,  2.2131273e-01, ...,
            5.2610713e-01, -6.2876856e-03, -6.4119155e-04],
          [ 1.2532182e-01,  3.6852699e-02,  2.1462679e-01, ...

In [ ]:
ckpt = {"model": state, "step": batch_count}


checkpoint_dir = f"checkpoints/{model_name}"
checkpoint_dir = os.path.abspath(checkpoint_dir)

# os.makedirs(checkpoint_dir, exist_ok=True)

orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
save_args = orbax_utils.save_args_from_target(ckpt)
orbax_checkpointer.save(checkpoint_dir, ckpt, save_args=save_args)

In [ ]:
new_checkpoint = orbax_checkpointer.restore(checkpoint_dir)
new_state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
new_state = new_state.replace(params=new_checkpoint["model"]["params"])

/Users/kailukowiak/Hephaestus/.venv/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1552: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [ ]:
numeric_inputs, categorical_inputs = train_ds[0]
numeric_inputs = jnp.array([numeric_inputs])
categorical_inputs = jnp.array([categorical_inputs])
numeric_out, categorical_out = state.apply_fn(
    {"params": state.params},
    # jnp.array(i[0]),
    # jnp.array(i[1]),
    numeric_inputs,
    categorical_inputs,
    deterministic=True,
    mask_data=False,
)
numeric_out

'numeric_out'

In [ ]:
@dataclass
class Results:
    numeric_out: jnp.array
    categorical_out: jnp.array
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array


def return_results(state, dataset, idx=0, mask_start: int = None):
    numeric_inputs, categorical_inputs = dataset[idx]
    if mask_start:
        numeric_inputs = numeric_inputs[:, :mask_start]
        categorical_inputs = categorical_inputs[:, :mask_start]
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    out = state.apply_fn(
        {"params": state.params},
        # hp.mask_tensor(jnp.array([train_ds[0]]), dataset, prng_key=key),
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs,
        deterministic=True,
        mask_data=MASK_DATA,
    )
    numeric_out, categorical_out = out["numeric_out"], out["categorical_out"]
    return Results(numeric_out, categorical_out, numeric_inputs, categorical_inputs)


x = return_results(state, train_ds, 0)
x.categorical_out

Array([[[[-1.6116713e-01, -6.4119155e-04, -4.7015983e-01, ...,
           1.2119155e+01, -6.4119155e-04, -6.4119155e-04],
         [-1.5990634e-01, -6.4119155e-04, -4.7382033e-01, ...,
           1.1891937e+01, -6.4119155e-04, -6.4119155e-04],
         [-1.4844951e-01, -6.4119155e-04, -4.7759899e-01, ...,
           1.1907718e+01, -6.4119155e-04, -6.4119155e-04],
         ...,
         [-3.2130098e-01, -6.4119155e-04, -4.0466940e-01, ...,
           1.0415320e+01, -6.4119155e-04, -6.4119155e-04],
         [-3.3229074e-01, -6.4119155e-04, -4.1116402e-01, ...,
           1.0454638e+01, -6.4119155e-04, -6.4119155e-04],
         [-3.3714718e-01, -6.4119155e-04, -4.1734508e-01, ...,
           1.0481203e+01, -6.4119155e-04, -6.4119155e-04]],

        [[ 1.0859615e-01, -1.8643760e-04,  1.7496361e-01, ...,
           9.8416910e+00, -1.8643760e-04, -1.8643760e-04],
         [ 1.0812584e-01, -1.8643760e-04,  1.7632909e-01, ...,
           1.0557760e+01, -1.8643760e-04, -1.8643760e-04],
        

In [ ]:
mask_data = False
MASK_DATA = True


def process_results(arr: jnp.array, col_names: list):
    arr = jnp.squeeze(arr)
    df = pd.DataFrame(arr.T)
    df.columns = col_names
    return df


@dataclass
class DFComparison:
    input_df: pd.DataFrame
    output_df: pd.DataFrame


def show_results_df(state, base_df, dataset, idx: int = 0, mask_start: int = None):
    results = return_results(state, dataset, idx=idx, mask_start=mask_start)

    input_categorical = process_results(
        results.categorical_inputs, dataset.categorical_col_tokens
    )
    input_numeric = process_results(results.numeric_inputs, dataset.numeric_col_tokens)
    output_categorical = process_results(
        results.categorical_out, dataset.categorical_col_tokens
    )
    output_numeric = process_results(results.numeric_out, dataset.numeric_col_tokens)
    input_df = pd.concat([input_categorical, input_numeric], axis=1)
    output_df = pd.concat([output_categorical, output_numeric], axis=1)

    return DFComparison(input_df, output_df)


df_comp = show_results_df(state, train_ds, train_ds, idx=0)
df_comp.input_df

ValueError: Must pass 2-d input. shape=(38, 59, 2)

In [ ]:
test_key = random.PRNGKey(4454)
x = jax.random.normal(test_key, (4, 26, 59, 256))

In [ ]:
no_mask_out = state.apply_fn(
    {"params": state.params},
    # jnp.array([test_ds[0][:10, :]]),
    jnp.array([test_ds[0][:, :10]]),
    deterministic=True,
    mask_data=True,
)
mask_out = state.apply_fn(
    {"params": state.params},
    jnp.array([test_ds[0][:, :20]]),
    deterministic=True,
    mask_data=True,
)
mask_out_df = pd.DataFrame(jnp.squeeze(mask_out).T)
mask_out_df.columns = test_df.columns[1:]
no_mask_out_df = pd.DataFrame(jnp.squeeze(no_mask_out).T)
no_mask_out_df.columns = test_df.columns[1:]

test_diff = mask_out_df - no_mask_out_df
test_diff

In [ ]:
def plot_planets(df_pred: pd.DataFrame, df_actual: pd.DataFrame, column: str, offset=0):
    plt.figure(figsize=(15, 10))
    plt.plot(df_pred[column], label="Autogregressive")
    plt.plot(df_actual[column], label="Actual")
    plt.title(f"{column} Predictions")
    plt.legend()
    # Show ticks and grid lines every 1 step
    plt.xticks(np.arange(0, len(df_pred), 1))
    plt.grid()
    # add black line at 0 on the y axis to show the difference
    plt.axhline(0, color="black")
    plt.show()

In [ ]:
def auto_regressive_predictions(
    state: train_state.TrainState, inputs: jnp.ndarray
) -> np.ndarray:
    # get the first row that contains all nan vales
    # if nan_rows_start >= stop_idx:
    #     return inputs
    nan_columns = jnp.isnan(inputs).all(axis=1)
    outputs = state.apply_fn(
        {"params": state.params},
        jnp.array([inputs]),
        # jnp.array([inputs]),
        deterministic=True,
        mask_data=MASK_DATA,
    )
    outputs = jnp.squeeze(outputs)
    final_row = np.array(outputs[:, -1])
    final_row = final_row[:, None]
    inputs = jnp.concatenate([inputs, final_row], axis=1)
    inputs = np.array(inputs)
    inputs[nan_columns] = np.nan
    return inputs
    # return auto_regressive_predictions(state, inputs, stop_idx)

In [ ]:
base_inputs = test_ds[300]
inputs_test = base_inputs[:, :10]
print(inputs_test.shape)
for i in trange(21):
    inputs_test = auto_regressive_predictions(state, inputs_test)

# x = auto_regressive_predictions(state, test_ds[0], 10)

In [ ]:
df_auto = pd.DataFrame(inputs_test.T)
df_actual = pd.DataFrame(base_inputs.T)
df_auto.columns = train_df.columns[1:]
df_actual.columns = train_df.columns[1:]
df_diff = df_auto - df_actual

# Drop rows that are all nan
df_diff = df_diff.dropna(axis=0, how="all")

In [ ]:
plot_planets(df_auto, df_actual, "time_step")

In [ ]:
plot_planets(df_auto, df_actual, "planet3_x")

In [ ]:
# inputs = jnp.array(next(iter(test_data_loader)))
inputs_test = make_batch(test_ds, 0, 1)

outputs = new_state.apply_fn(
    {"params": state.params},
    # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
    inputs_test,
    deterministic=True,
    mask_data=True,
)
df_actual = pd.DataFrame(jnp.squeeze(inputs_test).T)
df_actual.columns = test_df.columns[1:]

df_pred = pd.DataFrame(jnp.squeeze(outputs).T)
df_pred.columns = test_df.columns[1:]
plot_planets(df_pred, df_actual, "time_step")
plot_planets(df_pred, df_actual, "planet2_y")

In [ ]:
inputs_test.shape

In [ ]:
show_heatmap(df_diff, "Auto Regressive Predictions")

In [ ]:
# plot planet0_x from df_auto and df_actual


res = show_results_df(state, train_df, train_ds, idx=299, mask_start=20)
plot_planets(res["pred"], res["actual_masked"], "planet2_y", offset=0)
# plot_planets(df_auto, df_actual, "planet2_y")

In [ ]:
# plot planet0_x from df_auto and df_actual
res = show_results_df(state, train_df, train_ds, idx=20, mask_start=20)
plot_planets(res["pred"], res["actual_no_mask"], "planet1_y", offset=0)

In [ ]:
res = show_results_df(state, train_df, test_ds, idx=0, mask_start=30)

plot_planets(res["pred"], res["actual_masked"], "planet2_x", offset=0)

In [ ]:
loss, key = eval_step(state, jnp.array(next(iter(test_data_loader))), base_key)
loss, key